In [251]:
import pandas as pd
import requests
import pprint
import json
import time
from pandas import json_normalize
import ast

# 최대 출력 컬럼 제한 해제
pd.set_option('display.max_colwidth', None)

In [252]:
units_df = pd.read_csv('특성과 챔피언 컬럼 붙인 테이블.csv')

units_df.tail()

,gameId,game_datetime,game_length,game_version,puuid,placement,last_round,level,이타심,거대 괴수,...,쓰레쉬,초가스,말파이트,가렌,티모,자야,Combination,unit_based_cluster,traits_based_cluster,units_and_traits_cluster
9283,KR_7005139736,1711571877752,2173.421143,Version 14.6.570.6276 (Mar 25 2024/14:00:15) [PUBLIC] <Releases/14.6>,sMD7iMyWZ2ztcGvcYyXf92yYF6ui0ipF8UInDkKm6wMnJc7abC_rkU3WJokvCuiO4Rln3ltQCc3xTw,4,33,10,2,0,...,0,0,0,0,0,0,"TFT11_Riven, TFT11_Galio, TFT11_Kayle, TFT11_Soraka, TFT11_Diana, TFT11_Hwei, TFT11_Rakan, TFT11_WuKong, TFT11_Lissandra, TFT11_LeeSin, TFT11_Irelia",1,2,13
9284,KR_7005139736,1711571877752,2173.421143,Version 14.6.570.6276 (Mar 25 2024/14:00:15) [PUBLIC] <Releases/14.6>,4W5dMgFaI2qWXtzftLCjec16RypS_e_0BaHIynnzUApIEjpzmow5edpCgoq2Z7R4wwe8AhejYgf_sw,3,34,9,1,1,...,0,0,0,0,0,0,"TFT11_Udyr, TFT11_Ornn, TFT11_Azir, TFT11_Janna, TFT11_Soraka, TFT11_Diana, TFT11_Rakan, TFT11_WuKong, TFT11_Morgana",18,11,0
9285,KR_7005139736,1711571877752,2173.421143,Version 14.6.570.6276 (Mar 25 2024/14:00:15) [PUBLIC] <Releases/14.6>,m-WJmbLS17zaoDDdN1NjqRMGuXkZChReSv8-36lQ32zgt2-TJU4Noj7CY246rk6qA0WmYVOE4kTogQ,1,37,8,0,1,...,3,0,0,0,0,0,"TFT11_Lissandra, TFT11_Illaoi, TFT11_Ashe, TFT11_Lux, TFT11_Amumu, TFT11_Syndra, TFT11_Aphelios, TFT11_Thresh",0,4,3
9286,KR_7005139736,1711571877752,2173.421143,Version 14.6.570.6276 (Mar 25 2024/14:00:15) [PUBLIC] <Releases/14.6>,eJBg1EVkQRS4fXbZKDAkyuQuYzL-kF1Ps7NOxHuwiAQ29VZgBEi20Je7sbVtqV-m-Y6ArWw2Hck5Iw,6,28,8,0,0,...,0,0,0,0,0,0,"TFT11_Shen, TFT11_Caitlyn, TFT11_Jax, TFT11_Aatrox, TFT11_Senna, TFT11_Morgana, TFT11_Kayn, TFT11_Illaoi",6,9,4
9287,KR_7005139736,1711571877752,2173.421143,Version 14.6.570.6276 (Mar 25 2024/14:00:15) [PUBLIC] <Releases/14.6>,H7I8YjZ_H49ijjt7z4b6CZKJ0up1ug51YNS_PUhLR4p6cUAbQf0Vr91bpmtC2fQ4_DTZj_Hxl5OTlg,5,31,8,0,1,...,2,0,0,0,0,0,"TFT11_Illaoi, TFT11_Ashe, TFT11_Yorick, TFT11_Sett, TFT11_Amumu, TFT11_Syndra, TFT11_Aphelios, TFT11_Thresh",0,16,3


##### 각 컬럼의 1~3 을 1로 통일한 데이터 프레임으로 생성

In [253]:
##'시비르'부터 '자야'까지의 컬럼값이 0인 경우를 제외하고 모두 1로 변경
units_df_01 = units_df.copy()

units_df_01.loc[:, '시비르':'자야'] = units_df_01.loc[:, '시비르':'자야'].applymap(lambda x: 0 if x == 0 else 1)
              # 시비르 컬럼부터 자야 컬럼까지 모든행을 선택                         # 선택한 모든 데이터(applymap)에 람다 함수를 적용
                                                                                 # 여기서 람다 함수는 x가 0이면 0 나머지는 1로 치환

units_df_01.head(1)

,gameId,game_datetime,game_length,game_version,puuid,placement,last_round,level,이타심,거대 괴수,...,쓰레쉬,초가스,말파이트,가렌,티모,자야,Combination,unit_based_cluster,traits_based_cluster,units_and_traits_cluster
0,KR_7005435650,1711611092537,2339.676514,Version 14.6.570.6276 (Mar 25 2024/14:00:15) [PUBLIC] <Releases/14.6>,BWLn3nfAuki-3ju22n-ni5exBESPlXaPsyF9bZ3uFeuvr1ks-ZL6B7x3SA_KVt_kNrHRfsg6VVk9YQ,7,28,8,0,1,...,0,0,0,0,0,0,"TFT11_Sivir, TFT11_Riven, TFT11_Shen, TFT11_TahmKench, TFT11_Galio, TFT11_Kaisa, TFT11_Sylas, TFT11_Udyr, TFT11_Kayle",5,13,2


##### 각 챔피언의 등장 횟수 확인

In [254]:
## 각 챔피언 컬럼중 0이 아닌 값의 갯수 = 전체 조합 데이터에서 각 챔피언의 등장 횟수

# units_df의 해당 범위의 컬럼 중에서 0이 아닌 값들의 갯수를 구합니다.
units_count = pd.DataFrame((units_df_01.loc[:, '시비르':'자야'] != 0).sum().sort_values(ascending=False))

units_count.T

,다이애나,리븐,오공,라칸,소라카,케일,흐웨이,오른,갈리오,우디르,...,볼리베어,럭스,가렌,트리스타나,초가스,티모,나르,렉사이,자야,코부코
0,3074,2976,2950,2885,2775,2761,2641,2611,2426,2424,...,513,473,466,461,459,428,426,423,295,145


#### 직접 그룹바이를 통해 덱 분류하기 위해 덱분류 기준 찾기

기물의 유무나 시너지의 유무 조건을 조절하면서 적절한 기준을 찾기<br>
시너지에 관해서는 보다 엄격한 기준이 제공되어야하며<br>
챔피언의 경우 상징을 통한 유동성이 있을 수 있으므로, 덱의 아주 핵심이 되는 챔피언에 대해서만 기준을 정하는 것이 좋음<br>

##### 신화 바드 리롤 덱 분류 작업

In [255]:
# 바드, 탐 켄치, 조이, 소라카 포함 / 카이사 티모 미포함
bard_reroll = ((units_df['바드'] != 0) & 
               (units_df['탐 켄치'] != 0) & 
               (units_df['조이'] != 0)  & 
               (units_df['소라카'] != 0)& 
               (units_df['티모'] == 0) & 
               (units_df['카이사'] == 0)).sum()

# 결과 출력
bard_reroll

485

In [256]:
# 바드, 탐 켄치 포함 / 시너지 재주꾼 1 , 신화 1
bard_reroll = ((units_df['바드'] != 0) & 
               (units_df['탐 켄치'] != 0) & 
               (units_df['재주꾼'] == 1) & 
               (units_df['신화'] == 1)).sum()

# 결과 출력
bard_reroll

461

In [257]:
# 바드, 탐 켄치 포함 / 시너지 재주꾼 1 , 신화 1
bard_reroll = ((units_df['바드'] != 0) & 
               (units_df['탐 켄치'] != 0) & 
               (units_df['조이'] != 0) & 
               (units_df['소라카'] != 0) & 
               (units_df['니코'] != 0) & 
               (units_df['재주꾼'] == 1) & 
               (units_df['신화'] == 1) & 
               (units_df['이야기꾼'] == 1)).sum()

# 결과 출력
bard_reroll

381

In [258]:
# 바드, 탐 켄치, 소라카, 조이 포함 / 시너지 재주꾼 1 , 신화 1이상, 이야기꾼 1 이상
## 재주꾼 상징은 사실상 만들기 불가하므로 변동가능한 시너지에서 배제
## 첫번째 컨디션을 주었을 때와 숫자가 거의 유사하므로 채택

신화_바드_리롤 = ((units_df['바드'] != 0) & 
            (units_df['탐 켄치'] != 0) & 
            (units_df['조이'] != 0) & 
            (units_df['소라카'] != 0) & 
            (units_df['니코'] != 0) & 
            (units_df['재주꾼'] == 1) & 
            (units_df['신화'] != 0) & 
            (units_df['이야기꾼'] != 0)).sum()

# 결과 출력
신화_바드_리롤

479

##### 6난동꾼 덱

In [259]:
## 난동꾼 6시너지 이상
난동꾼 = ((units_df['난동꾼'] >= 3)).sum()

# 결과 출력
난동꾼

78

##### 난동 카이사 덱 분류 - 6난동 4재주꾼이어도 난동 카이사로 분류

In [260]:
## 카이사, 티모, 갈리오가 존재 / 재주꾼 4시너지 , 난동꾼 4시너지
난동_카이사 = ((units_df['카이사'] != 0) & 
          (units_df['티모'] != 0) & 
          (units_df['갈리오'] != 0) & 
          (units_df['재주꾼'] == 2) & 
          (units_df['난동꾼'] == 2) & 
          (units_df['이야기꾼'] != 0)).sum()

# 결과 출력
난동_카이사

229

##### 리롤 결투가 덱 분류

In [261]:
## 트리스타나, 볼리베어 2성 이상 / 결투가 3시너지 이상
리롤_결투가 = ((units_df['볼리베어'] >= 2) & 
          (units_df['트리스타나'] >= 2) & 
          (units_df['결투가'] >= 3)).sum()

# 결과 출력
리롤_결투가

364

##### 8 결투가 덱 분류 _ 트타, 볼베가 3성이더라도 곁투가 시너지가 8이면 8결투가로 분류

In [262]:
## 결투가 8시너지
시너지8_결투가 = ((units_df['결투가'] >= 4)).sum()

# 결과 출력
시너지8_결투가

85

##### 7 이야기꾼 덱 분류

In [263]:
## 카이사, 티모, 갈리오가 존재 / 재주꾼 4시너지 , 난동꾼 4시너지
시너지7_이야기꾼 = ((units_df['이야기꾼'] >= 3)).sum()

# 결과 출력
시너지7_이야기꾼

360

##### 6혼령 리롤덱 분류

In [264]:
## 세나, 쉔을 포함하면서 6혼령이 켜진덱
혼령6_리롤 = ((units_df['세나'] != 0) & 
          (units_df['쉔'] != 0) & 
          (units_df['혼령'] >= 3)).sum()

# 결과 출력
혼령6_리롤

395

##### 7필연덱 분류

In [265]:
## 7필연 시너지가 켜진 덱
필연7 = ((units_df['필연'] >= 3)).sum()

# 결과 출력
필연7

402

##### 필연 야스오 아리 리롤 분류 - 데이터 갯수 없어서 안해도됨

In [266]:
## 야스오, 아리가 3성이고 필연 시너지가 5이상 켜진 덱
필연7 = ((units_df['야스오'] == 3) & (units_df['아리'] == 0) & (units_df['필연'] >= 2)).sum()

# 결과 출력
필연7

6

##### 리롤 나르 세나 분류

In [267]:
## 세나, 나르를 포함하면서 숲지기 시너지를 켜준 덱
나르_세나_리롤 = ((units_df['세나'] != 0) & (units_df['나르'] != 0) & (units_df['숲지기'] != 0)).sum()

# 결과 출력
나르_세나_리롤

280

##### 6 암영 4사신 요네 덱 분류

In [268]:
## 요네를 포함하면서 암영 6시너지 사신 4시너지가 켜진 덱
암영6_사신4 = ((units_df['요네'] >= 2) & (units_df['암영'] == 3)& (units_df['사신'] == 2)).sum()

# 결과 출력
암영6_사신4

93

##### 6 천계덱 분류 (아마 천계 즈롯일듯)

In [269]:
## 천계 6시너지 이상이 켜진 덱
천계 = ((units_df['천계'] >= 5)).sum()

# 결과 출력
천계

209

##### 초코말케 리롤덱

In [270]:
## 초가스, 코그모, 말파이트, 케이틀린을 포함하면서 코그모 3성을 찍어준 덱
초코말케_리롤 = ((units_df['코그모'] == 3) & (units_df['케이틀린'] != 0) & (units_df['초가스'] != 0)).sum()

# 결과 출력
초코말케_리롤

274

##### 7신화 릴리아

In [271]:
## 7신화 시너지를 켜주로 릴리아가 들어간덱, 초코말케덱 제외를 위해 케이틀린이 없는 덱만 지정
신화_릴리아 = ((units_df['릴리아'] == 2) & (units_df['신화'] >= 3) & (units_df['케이틀린'] == 0)).sum()

# 결과 출력
신화_릴리아

106

##### 아펠 아무무 리롤덱

In [272]:
## 아펠, 아무무, 일라오이, 애쉬가 모두 있는 덱
아펠_아무무_리롤 = ((units_df['아펠리오스'] >= 2) & (units_df['아무무'] >= 2) & (units_df['일라오이'] >= 2) & (units_df['애쉬'] >= 1) & (units_df['필연'] >= 1) & (units_df['저격수'] == 1)).sum()

# 결과 출력
아펠_아무무_리롤

310

##### 6도자기 4파수꾼 - 데이터 갯수가 적어서 안하는 것도 고려 - 제외

In [273]:
## 6도자기 4파수꾼 시너지를 켜준덱
도가지6_파수꾼4 = ((units_df['도자기'] >= 3) & (units_df['파수꾼'] >= 2)).sum()

# 결과 출력
도가지6_파수꾼4

11

##### 6기원자 덱 분류

In [274]:
## 6기원자 시너지가 켜지면서 릴리아 애니가 들어간덱
기원자6 = ((units_df['기원자'] >= 3) & (units_df['애니'] != 0) & (units_df['릴리아'] != 0)).sum()

# 결과 출력
기원자6

173

##### 4 현자 모르가나

In [275]:
## 모르가나, 갈리오, 소라카, 케인, 리븐, 자이라 가 들어간덱
현자4_모르가나 = ((units_df['모르가나'] != 0) & (units_df['갈리오'] >= 1) & (units_df['현자'] >= 2)).sum()

## 결과 출력
현자4_모르가나

457

##### 4 용군주 밸류 덱

In [276]:
## 용군주 시너지가 켜진 덱중 오른이 들어간 덱
용군주4_밸류 = ((units_df['용군주'] >= 3)).sum()

# 결과 출력
용군주4_밸류

847

##### 용군주 흐웨이 밸류덱 분석 - 성공덱과 실패덱으로 분류

In [277]:
## 용군주 시너지가 켜진 덱중 오른이 들어간 덱
용군주_흐웨이_시도 = ((units_df['용군주'] >= 2) & (units_df['현자'] >= 1)).sum()

# 결과 출력
용군주_흐웨이_시도

1494

In [278]:
## 우디르, 흐웨이, 오공, 다이애나, 우디르가 모두 들어간덱, 용군주 시너지가 3이상 켜진덱
용군주_흐웨이_성공 = ((units_df['우디르'] != 0) & (units_df['흐웨이'] >= 1) & (units_df['오공'] != 0) & (units_df['다이애나'] != 0)& (units_df['라칸'] != 0)& (units_df['용군주'] >= 1)& (units_df['아지르'] != 0)).sum()

용군주_흐웨이_성공

486

##### 6비전 마법사 덱

In [279]:
## 6비전 마법사 시너지가 켜진덱
비전마법사6 = ((units_df['비전 마법사'] >= 3)).sum()

# 결과 출력
비전마법사6

131

##### 4파수꾼 4비전 마법사

In [280]:
## 4비전 마법사 4 파수꾼 시너지가 켜진덱
파수꾼4_비전마법사4 = ((units_df['비전 마법사'] >= 2) & (units_df['파수꾼'] >= 2)).sum()

# 결과 출력
파수꾼4_비전마법사4

55

##### 5먹그림자 카이사

In [281]:
## 카이사가 있고 5먹그림자 시너지가 켜진덱
먹그림자5_카이사 = ((units_df['먹그림자'] >= 2) & (units_df['카이사'] >= 1)).sum()

# 결과 출력
먹그림자5_카이사

49

##### 6거대괴수덱

In [282]:
## 거대괴수 6시너지 이상
거대괴수 = ((units_df['거대 괴수'] >= 3)).sum()

# 결과 출력
거대괴수

38

#### 앞에서 만든 덱 기준으로 데이터 라벨링 (덱 분류)

In [283]:
deck_classify = units_df.copy()

deck_classify.head(1)

,gameId,game_datetime,game_length,game_version,puuid,placement,last_round,level,이타심,거대 괴수,...,쓰레쉬,초가스,말파이트,가렌,티모,자야,Combination,unit_based_cluster,traits_based_cluster,units_and_traits_cluster
0,KR_7005435650,1711611092537,2339.676514,Version 14.6.570.6276 (Mar 25 2024/14:00:15) [PUBLIC] <Releases/14.6>,BWLn3nfAuki-3ju22n-ni5exBESPlXaPsyF9bZ3uFeuvr1ks-ZL6B7x3SA_KVt_kNrHRfsg6VVk9YQ,7,28,8,0,1,...,0,0,0,0,0,0,"TFT11_Sivir, TFT11_Riven, TFT11_Shen, TFT11_TahmKench, TFT11_Galio, TFT11_Kaisa, TFT11_Sylas, TFT11_Udyr, TFT11_Kayle",5,13,2


In [284]:
#### 필요없는 컬럼 드랍
deck_classify.drop(columns=['unit_based_cluster', 'traits_based_cluster', 'units_and_traits_cluster'], inplace=True)

deck_classify.head(1)

,gameId,game_datetime,game_length,game_version,puuid,placement,last_round,level,이타심,거대 괴수,...,코부코,다리우스,아펠리오스,쓰레쉬,초가스,말파이트,가렌,티모,자야,Combination
0,KR_7005435650,1711611092537,2339.676514,Version 14.6.570.6276 (Mar 25 2024/14:00:15) [PUBLIC] <Releases/14.6>,BWLn3nfAuki-3ju22n-ni5exBESPlXaPsyF9bZ3uFeuvr1ks-ZL6B7x3SA_KVt_kNrHRfsg6VVk9YQ,7,28,8,0,1,...,0,0,0,0,0,0,0,0,0,"TFT11_Sivir, TFT11_Riven, TFT11_Shen, TFT11_TahmKench, TFT11_Galio, TFT11_Kaisa, TFT11_Sylas, TFT11_Udyr, TFT11_Kayle"


##### 덱 이름을 적어줄 컬럼 생성

In [285]:
deck_classify['deck_name'] = '분류 불가'

deck_classify.head(1)

,gameId,game_datetime,game_length,game_version,puuid,placement,last_round,level,이타심,거대 괴수,...,다리우스,아펠리오스,쓰레쉬,초가스,말파이트,가렌,티모,자야,Combination,deck_name
0,KR_7005435650,1711611092537,2339.676514,Version 14.6.570.6276 (Mar 25 2024/14:00:15) [PUBLIC] <Releases/14.6>,BWLn3nfAuki-3ju22n-ni5exBESPlXaPsyF9bZ3uFeuvr1ks-ZL6B7x3SA_KVt_kNrHRfsg6VVk9YQ,7,28,8,0,1,...,0,0,0,0,0,0,0,0,"TFT11_Sivir, TFT11_Riven, TFT11_Shen, TFT11_TahmKench, TFT11_Galio, TFT11_Kaisa, TFT11_Sylas, TFT11_Udyr, TFT11_Kayle",분류 불가


##### 각 조건을 이용하여 덱에 적합한 이름을 라벨링

In [286]:
## 신화 바드 리롤 덱 라벨링
deck_classify.loc[((units_df['바드'] != 0) & 
                   (units_df['탐 켄치'] != 0) & 
                   (units_df['조이'] != 0) & 
                   (units_df['소라카'] != 0) & 
                   (units_df['티모'] == 0) & 
                   (units_df['카이사'] == 0)), 
                  'deck_name'] = '신화 바드 리롤'

# 확인
check_deck_name = deck_classify[deck_classify['deck_name'] == '신화 바드 리롤']

check_deck_name[['Combination', 'deck_name']].T

,15,23,29,38,42,56,96,148,173,180,...,9159,9160,9168,9169,9176,9186,9193,9196,9213,9242
Combination,"TFT11_Sivir, TFT11_Riven, TFT11_TahmKench, TFT11_Kayle, TFT11_Soraka, TFT11_Rakan, TFT11_Neeko, TFT11_Bard, TFT11_Zoe","TFT11_Sivir, TFT11_Riven, TFT11_TahmKench, TFT11_Kayle, TFT11_Soraka, TFT11_Neeko, TFT11_Bard, TFT11_Zoe","TFT11_Sivir, TFT11_Riven, TFT11_TahmKench, TFT11_Kayle, TFT11_Soraka, TFT11_Hwei, TFT11_Neeko, TFT11_Bard, TFT11_Zoe, TFT11_Nautilus","TFT11_Sivir, TFT11_Riven, TFT11_TahmKench, TFT11_Kayle, TFT11_Soraka, TFT11_Rakan, TFT11_WuKong, TFT11_Neeko, TFT11_Bard, TFT11_Zoe","TFT11_Sivir, TFT11_Riven, TFT11_TahmKench, TFT11_Kayle, TFT11_Soraka, TFT11_Rakan, TFT11_Lissandra, TFT11_Neeko, TFT11_Bard, TFT11_Zoe","TFT11_Sivir, TFT11_Riven, TFT11_TahmKench, TFT11_Kayle, TFT11_Soraka, TFT11_Rakan, TFT11_WuKong, TFT11_Lissandra, TFT11_Neeko, TFT11_Bard, TFT11_Zoe","TFT11_Sivir, TFT11_Riven, TFT11_TahmKench, TFT11_Kayle, TFT11_Soraka, TFT11_Hwei, TFT11_Lissandra, TFT11_Neeko, TFT11_Bard, TFT11_Zoe, TFT11_Nautilus","TFT11_Sivir, TFT11_Riven, TFT11_TahmKench, TFT11_Kayle, TFT11_Soraka, TFT11_Hwei, TFT11_Rakan, TFT11_WuKong, TFT11_Neeko, TFT11_Bard, TFT11_Zoe","TFT11_Sivir, TFT11_Riven, TFT11_TahmKench, TFT11_Kayle, TFT11_Soraka, TFT11_WuKong, TFT11_Neeko, TFT11_Bard, TFT11_Zoe","TFT11_Sivir, TFT11_Riven, TFT11_TahmKench, TFT11_Kayle, TFT11_Soraka, TFT11_Rakan, TFT11_Lissandra, TFT11_Neeko, TFT11_Bard, TFT11_Zoe, TFT11_ChoGath",...,"TFT11_Sivir, TFT11_Riven, TFT11_TahmKench, TFT11_Kayle, TFT11_Soraka, TFT11_Neeko, TFT11_Bard, TFT11_Zoe, TFT11_Nautilus","TFT11_Sivir, TFT11_Riven, TFT11_TahmKench, TFT11_Kayle, TFT11_Soraka, TFT11_Neeko, TFT11_Bard, TFT11_Zoe","TFT11_Sivir, TFT11_TahmKench, TFT11_Galio, TFT11_Sylas, TFT11_Kayle, TFT11_RekSai, TFT11_Soraka, TFT11_Neeko, TFT11_Bard, TFT11_Zoe","TFT11_Sivir, TFT11_Riven, TFT11_TahmKench, TFT11_Kayle, TFT11_Soraka, TFT11_WuKong, TFT11_Lissandra, TFT11_Neeko, TFT11_Bard, TFT11_Zoe","TFT11_Sivir, TFT11_Riven, TFT11_TahmKench, TFT11_Kayle, TFT11_Soraka, TFT11_Neeko, TFT11_Bard, TFT11_Zoe","TFT11_Sivir, TFT11_Riven, TFT11_TahmKench, TFT11_Kayle, TFT11_Soraka, TFT11_Rakan, TFT11_Neeko, TFT11_Bard, TFT11_Zoe, TFT11_Xayah","TFT11_Sivir, TFT11_Riven, TFT11_TahmKench, TFT11_Kayle, TFT11_Soraka, TFT11_Neeko, TFT11_Bard, TFT11_Zoe, TFT11_Qiyana","TFT11_Sivir, TFT11_Riven, TFT11_TahmKench, TFT11_Kayle, TFT11_Soraka, TFT11_Rakan, TFT11_Neeko, TFT11_Bard, TFT11_Zoe","TFT11_Sivir, TFT11_Riven, TFT11_TahmKench, TFT11_Kayle, TFT11_Soraka, TFT11_Neeko, TFT11_Bard, TFT11_Zoe, TFT11_Nautilus","TFT11_Sivir, TFT11_Riven, TFT11_TahmKench, TFT11_Kayle, TFT11_Soraka, TFT11_Rakan, TFT11_WuKong, TFT11_Neeko, TFT11_Bard, TFT11_Zoe"
deck_name,신화 바드 리롤,신화 바드 리롤,신화 바드 리롤,신화 바드 리롤,신화 바드 리롤,신화 바드 리롤,신화 바드 리롤,신화 바드 리롤,신화 바드 리롤,신화 바드 리롤,...,신화 바드 리롤,신화 바드 리롤,신화 바드 리롤,신화 바드 리롤,신화 바드 리롤,신화 바드 리롤,신화 바드 리롤,신화 바드 리롤,신화 바드 리롤,신화 바드 리롤


In [287]:
## 6난동꾼
deck_classify.loc[(units_df['난동꾼'] >= 3), 'deck_name'] = '6난동꾼'


## 난동 카이사 
deck_classify.loc[((units_df['카이사'] != 0) & 
                   (units_df['티모'] != 0) & 
                   (units_df['갈리오'] != 0) & 
                   (units_df['재주꾼'] == 2) & 
                   (units_df['난동꾼'] == 2) & 
                   (units_df['이야기꾼'] != 0)), 
                  'deck_name'] = '난동꾼 카이사'


## 리롤 결투가
deck_classify.loc[((units_df['볼리베어'] >= 2) & 
                   (units_df['트리스타나'] >= 2) & 
                   (units_df['결투가'] >= 3)),
                   'deck_name'] = '리롤 결투가'


## 8결투가
deck_classify.loc[(units_df['결투가'] >= 4), 'deck_name'] = '8결투가'


## 7이야기꾼 
deck_classify.loc[(units_df['이야기꾼'] >= 3), 'deck_name'] = '7이야기꾼'


## 6혼령 리롤
deck_classify.loc[((units_df['세나'] >= 2) & 
                   (units_df['쉔'] >= 2) & 
                   (units_df['혼령'] >= 3)), 'deck_name'] = '6혼령 리롤'


## 7필연
deck_classify.loc[(units_df['필연'] >= 3), 'deck_name'] = '7필연'


## 나르 세나 리롤 
deck_classify.loc[((units_df['세나'] >= 2) & 
                   (units_df['나르'] >= 2) & 
                   (units_df['숲지기'] != 0)), 'deck_name'] = '나르 세나 리롤'

## 6암영 4사신 요네
deck_classify.loc[((units_df['요네'] >= 2) & 
                   (units_df['암영'] >= 3)& 
                   (units_df['사신'] >= 2)), 
                   'deck_name'] = '6암영 4사신 요네'


## 천계 
deck_classify.loc[(units_df['천계'] >= 5), 'deck_name'] = '천계'


## 신화 코그모 리롤 (초코말케) 
deck_classify.loc[((units_df['코그모'] == 3) & 
                   (units_df['말파이트'] != 0) & 
                   (units_df['케이틀린'] != 0) & 
                   (units_df['초가스'] != 0)), 
                   'deck_name'] = '신화 코그모 리롤'


## 7신화 릴리아 
deck_classify.loc[(units_df['릴리아'] >= 2) & 
                  (units_df['신화'] >= 3), 
                   'deck_name'] = '7신화 릴리아'


## 아펠 아무무 리롤 
deck_classify.loc[((units_df['아펠리오스'] >= 2) &  
                  (units_df['아무무'] >= 2) & 
                  (units_df['일라오이'] >= 2) & 
                  (units_df['애쉬'] >= 1) & 
                  (units_df['필연'] >= 1) & 
                  (units_df['저격수'] == 1)),'deck_name'] = '아펠 아무무 리롤'


## 6기원자 
deck_classify.loc[((units_df['기원자'] >= 3) & 
                   (units_df['애니'] != 0) & 
                    (units_df['릴리아'] != 0)),'deck_name'] = '6기원자'


## 현자 모르가나 
deck_classify.loc[((units_df['모르가나'] != 0) & 
                   (units_df['갈리오'] >= 1) & 
                   (units_df['현자'] >= 2)),'deck_name'] = '현자 모르가나'


## 용군주 흐웨이 밸류 시도
deck_classify.loc[((units_df['용군주'] >= 1) & 
              (units_df['오른'] != 0)), 'deck_name'] = '용군주 흐웨이 밸류 시도'


## 4 용군주 밸류
deck_classify.loc[(units_df['용군주'] >= 3) & (units_df['현자'] >= 1), 'deck_name'] = '4용군주 밸류'


## 용군주 흐웨이 밸류 완성/성공
deck_classify.loc[((units_df['우디르'] != 0) & 
              (units_df['흐웨이'] >= 1) & 
              (units_df['오공'] != 0) & 
              (units_df['다이애나'] != 0)& 
              (units_df['라칸'] != 0)& 
              (units_df['용군주'] >= 1)& 
              (units_df['아지르'] != 0)), 'deck_name'] = '용군주 흐웨이 밸류 완성'


## 6비전 마법사
deck_classify.loc[(units_df['비전 마법사'] >= 3), 'deck_name'] = '6비전 마법사'


## 4비전마법사 4파수꾼
deck_classify.loc[((units_df['비전 마법사'] >= 2) & 
               (units_df['파수꾼'] >= 2)), 'deck_name'] = '4비전마법사 4파수꾼'

## 5먹그림자 카이사
deck_classify.loc[((units_df['먹그림자'] >= 2) & 
             (units_df['카이사'] >= 1)), 'deck_name'] = '5먹그림자 카이사'

## 6거대괴수
deck_classify.loc[(units_df['거대 괴수'] >= 3), 'deck_name'] = '6거대괴수'


## 숲지기 킨드레드 리롤
deck_classify.loc[(units_df['숲지기'] >= 2) & (units_df['킨드레드'] >= 2) & (units_df['사신'] >= 2), 'deck_name'] = '숲지기 킨드레드 리롤'


# 확인
check_deck_name = deck_classify[deck_classify['deck_name'] == '5먹그림자 카이사']

check_deck_name[['Combination', 'deck_name']].T


,432,566,750,920,1378,1549,1955,2141,2260,2358,...,7891,7908,8192,8222,8577,8592,8818,8918,9031,9084
Combination,"TFT11_Kaisa, TFT11_Udyr, TFT11_Jax, TFT11_Aatrox, TFT11_Senna, TFT11_Ornn, TFT11_Irelia, TFT11_Volibear, TFT11_Sett, TFT11_Xayah","TFT11_Kaisa, TFT11_Udyr, TFT11_Caitlyn, TFT11_RekSai, TFT11_Jax, TFT11_Aatrox, TFT11_Senna, TFT11_Gnar, TFT11_Ornn, TFT11_Azir","TFT11_Shen, TFT11_Kaisa, TFT11_Sylas, TFT11_Udyr, TFT11_Aatrox, TFT11_Senna, TFT11_Morgana, TFT11_Kayn, TFT11_Illaoi","TFT11_Kaisa, TFT11_Udyr, TFT11_Jax, TFT11_Aatrox, TFT11_Senna, TFT11_Hwei, TFT11_Morgana, TFT11_Volibear, TFT11_Xayah","TFT11_Kaisa, TFT11_Udyr, TFT11_Jax, TFT11_Ornn, TFT11_Irelia, TFT11_Illaoi, TFT11_Volibear, TFT11_Amumu","TFT11_Riven, TFT11_Shen, TFT11_Kaisa, TFT11_Jax, TFT11_Aatrox, TFT11_Irelia, TFT11_Illaoi, TFT11_Volibear, TFT11_Xayah","TFT11_Shen, TFT11_Kaisa, TFT11_Udyr, TFT11_Caitlyn, TFT11_Aatrox, TFT11_Senna, TFT11_Morgana, TFT11_Kayn, TFT11_Illaoi","TFT11_Kaisa, TFT11_Caitlyn, TFT11_RekSai, TFT11_Jax, TFT11_Aatrox, TFT11_Senna, TFT11_Gnar, TFT11_Volibear","TFT11_Shen, TFT11_Kaisa, TFT11_Udyr, TFT11_Caitlyn, TFT11_Jax, TFT11_Aatrox, TFT11_Senna, TFT11_Morgana, TFT11_Illaoi","TFT11_Shen, TFT11_Kaisa, TFT11_Udyr, TFT11_Caitlyn, TFT11_Jax, TFT11_Aatrox, TFT11_Senna, TFT11_Illaoi",...,"TFT11_Kaisa, TFT11_Caitlyn, TFT11_RekSai, TFT11_Jax, TFT11_Aatrox, TFT11_Senna, TFT11_Gnar, TFT11_Ornn, TFT11_Azir, TFT11_Volibear","TFT11_Kaisa, TFT11_Udyr, TFT11_Caitlyn, TFT11_RekSai, TFT11_Jax, TFT11_Aatrox, TFT11_Senna, TFT11_Gnar, TFT11_Ornn","TFT11_Kaisa, TFT11_Udyr, TFT11_Caitlyn, TFT11_Jax, TFT11_Aatrox, TFT11_Senna, TFT11_Morgana, TFT11_Illaoi","TFT11_Shen, TFT11_Kaisa, TFT11_Udyr, TFT11_Caitlyn, TFT11_Aatrox, TFT11_Senna, TFT11_Morgana, TFT11_Illaoi, TFT11_Sett","TFT11_Shen, TFT11_Kaisa, TFT11_Udyr, TFT11_Caitlyn, TFT11_Aatrox, TFT11_Senna, TFT11_Morgana, TFT11_Kayn, TFT11_Illaoi","TFT11_Riven, TFT11_Kaisa, TFT11_Sylas, TFT11_Jax, TFT11_Aatrox, TFT11_Senna, TFT11_Illaoi, TFT11_Volibear","TFT11_Kaisa, TFT11_Sylas, TFT11_Jax, TFT11_Aatrox, TFT11_Senna, TFT11_Gnar, TFT11_Volibear, TFT11_Teemo","TFT11_Sivir, TFT11_Galio, TFT11_Kaisa, TFT11_Udyr, TFT11_Jax, TFT11_Aatrox, TFT11_WuKong, TFT11_Volibear","TFT11_Shen, TFT11_Kaisa, TFT11_Caitlyn, TFT11_Jax, TFT11_Aatrox, TFT11_Senna, TFT11_Illaoi, TFT11_Volibear","TFT11_Shen, TFT11_Kaisa, TFT11_Udyr, TFT11_Caitlyn, TFT11_Jax, TFT11_Aatrox, TFT11_Senna, TFT11_Diana, TFT11_Morgana, TFT11_Kayn, TFT11_Illaoi"
deck_name,5먹그림자 카이사,5먹그림자 카이사,5먹그림자 카이사,5먹그림자 카이사,5먹그림자 카이사,5먹그림자 카이사,5먹그림자 카이사,5먹그림자 카이사,5먹그림자 카이사,5먹그림자 카이사,...,5먹그림자 카이사,5먹그림자 카이사,5먹그림자 카이사,5먹그림자 카이사,5먹그림자 카이사,5먹그림자 카이사,5먹그림자 카이사,5먹그림자 카이사,5먹그림자 카이사,5먹그림자 카이사


In [288]:
deck_classify.to_csv('덱 분류 완료 테이블.csv', index=False)

In [289]:
deck_classify['deck_name'].nunique()

25